### General Imports

In [2]:
import pandas as pd
import numpy as np
from datetime import datetime as dt
import os
import glob
from cleanUp import cleanUp
from fillDf import fillDf


### Data Cleaning
Passing the sensor data through the cleanUp function to get fix timestamps and delete null timestamps.

In [3]:
all_csv_files = glob.glob("./Data/*.txt")
# insert the desired start time
cutOffTime = '12/22/2020 12:49'
# insert the time rectifying offsets. default of for nothing {'':0}
sensorConditions = {'BU':8,'S-':1}
columns = [0,1,6,7,8,9,10,11]

In [4]:
data = cleanUp(cutOffTime,sensorConditions,all_csv_files,columns)

S-01     2020-12-22 12:49:10      2020-12-22 16:09:31
S-02  NO DATA PRESENT    NO DATA PRESENT
S-03     2020-12-22 12:49:03      2020-12-22 16:09:35
S-04     2020-12-22 12:49:07      2020-12-22 16:09:34
S-05     2020-12-22 12:49:01      2020-12-22 16:10:50
S-06     2020-12-22 12:49:12      2020-12-22 16:09:25
S-07     2020-12-22 12:49:07      2020-12-22 16:12:53
S-08     2020-12-22 12:49:24      2020-12-22 16:57:24
S-09     2020-12-22 12:49:00      2020-12-22 16:11:30
S-11     2020-12-22 13:02:00      2020-12-22 16:09:43
S-12     2020-12-22 12:49:06      2020-12-22 16:10:06
S-13     2020-12-22 12:49:06      2020-12-22 16:08:44
S-14     2020-12-22 12:49:09      2020-12-22 16:13:45
S-15     2020-12-22 12:49:03      2020-12-22 16:10:00
S-BU1     2020-12-22 12:49:00      2020-12-22 16:11:40
S-BU2     2020-12-22 12:49:00      2020-12-22 16:10:30


### Exporting Data
Here we can export the organized data frames as csv files

In [5]:
for x in data:
    temp=data[x]
    location = os.path.join('./proccessedData',x+'.csv')
    temp.to_csv(location,index=False)

### Checking Data
Here we scan through the data for irregularities in data recording.

In [6]:
errors = {}
errorCount = {}
# Enter the expected interval here
interval = 10
for x in data:
    # errors keeps track of length of each time interval error that occurs
    errors[x] = set(())
    # errorCount keeps track of how many times each time interval error occured
    errorCount[x] = {}
    # counter keeps track of the total time interval errors per sensor
    counter = 0
    temp = data[x]
    for idx,i in enumerate(temp['Date_Time']):
        try:
            if not ((temp['Date_Time'][idx+1] - i) == pd.Timedelta(seconds=interval)):
                timeErr = temp['Date_Time'][idx+1] - i
                if str(timeErr.seconds) in errorCount[x]:
                    errorCount[x][str(timeErr.seconds)] +=1
                else:
                    errorCount[x][str(timeErr.seconds)] = 1

                errors[x].add(timeErr)


                counter += 1
        except:
            continue

    print(str(counter),' possible errors in ', x)
    # display the different types of errors
    lst = [i.seconds for i in errors[x]]
    frmt = "{:>4}"*len(lst)
    print(frmt.format(*lst))
    # display the quantity of each type of error
    lst = [errorCount[x][str(i.seconds)] for i in errors[x]]
    frmt = "{:>4}"*len(lst)
    print(frmt.format(*lst))
    print()

208  possible errors in  S-01
  26  21  17  13  19  30  20  11
   1   1   6   1   2   1 195   1

0  possible errors in  S-02



219  possible errors in  S-03
  21  12  25  27  16  18  62  84  51  20  31  11  22   9  13   0  90  15  70  59  17  19  63
   1   2   1   2   1   7   2   1   1 171   1   3   2   1   1   1   1   1   1   1  12   4   1

220  possible errors in  S-04
  21  22  17  23  60  18  35  24  78  19  20  13  25  11  49
   1   2  12   1   1  10   3   1   1   2 180   1   1   3   1

220  possible errors in  S-05
  26  21  16  17  23  12  18   7  19  24  25  20  15
   1   4   1  10   2   1   7   1   2   1   1 188   1

199  possible errors in  S-06
  28  20  85
   1 197   1

212  possible errors in  S-07
   9  16  22  14  12  34  18  24  19  30  25  20  15  17  13  11
   1   1   2   1   1   1   1   1   9   2   1 184   2   2   1   2

412  possible errors in  S-08
  34  35 138 139 140  37
 164 241   3   2   1   1

5  possible errors in  S-09
   9  17   3  31  20
   1   1   1   1 

Notice there are quite a few repeating errors here in our data set. We can either choose to interpolate the data inbetween or pad it with 0s. For gaps <40s i will interpolate, but for gaps >40 i will 0 pad.

In [7]:
interpDF = {}

for x in data:
    df = data[x]
    cutoff = 40
    freq = '10S'
    try:
        interpDF[x],accuracy = fillDf(df,freq,'12/22/2020 12:59:00','12/22/2020 16:10:00',cutoff)
        print(x,' ',accuracy)
    except IndexError:
        print(x,'NO DATA')

S-01   ['% of values from interpolation : 34.266', '% of values from 0-padding : 0.0', '% of values not changed : 65.734']
S-02   NO DATA
S-03   ['% of values from interpolation : 33.304', '% of values from 0-padding : 4.633', '% of values not changed : 62.063']
S-04   ['% of values from interpolation : 35.052', '% of values from 0-padding : 1.661', '% of values not changed : 63.287']
S-05   ['% of values from interpolation : 35.428', '% of values from 0-padding : 0.0', '% of values not changed : 64.572']
S-06   ['% of values from interpolation : 33.158', '% of values from 0-padding : 0.7', '% of values not changed : 66.142']
S-07   ['% of values from interpolation : 33.857', '% of values from 0-padding : 0.0', '% of values not changed : 66.143']
S-08   ['% of values from interpolation : 93.881', '% of values from 0-padding : 6.119', '% of values not changed : 0.0']
S-09   ['% of values from interpolation : 0.436', '% of values from 0-padding : 0.0', '% of values not changed : 99.564']

### Export Data
export the newly interpolated data

In [8]:
for x in interpDF:
    temp=interpDF[x]
    location = os.path.join('./interpolatedData',x+'.csv')
    temp.to_csv(location,index=False)
print(location)

./interpolatedData\S-BU2.csv


### Merge the DataFrames
Also remove 'S-02' from the dictionary as it has no real data
and find the least common index

In [9]:
interpDF.pop('S-02',None)
# interpDF.pop('S-BU2',None)
# interpDF.pop('S-BU1',None)
length = []
for x in interpDF:
    length.append(len(interpDF[x]))
index = min(length)
print(index)

1139


In [10]:
for count,key in enumerate(list(interpDF.keys())):
    print(count,key)

0 S-01
1 S-03
2 S-04
3 S-05
4 S-06
5 S-07
6 S-08
7 S-09
8 S-11
9 S-12
10 S-13
11 S-14
12 S-15
13 S-BU1
14 S-BU2


In [11]:
dfMerged = []
columns = list(interpDF.keys())
columns.extend(['Average',
'Variance',
'Zone 1',
'Var Z1',
'Zone 2',
'Var Z2',
'Zone 3',
'Var Z3',
'Zone 4',
'Var Z4'])

for idx,i in enumerate(interpDF[columns[0]].values[:index]):
    temp = []
    temp.append(i[0])
    for x in interpDF:
        temp.append(interpDF[x].values[idx][1])
    #here we add the overall average and variance columns
    temp.append(np.average(temp[1:]))
    temp.append(np.std(temp[1:]))
    #here we're segregating the zones in the file giving their variance and avg

    #Zone 1 the 2 sensors right on top of the nebulizer
    lst = temp[10:12]
    temp.append(np.average(lst))
    temp.append(np.std(lst))
    #Zone 2 the perimiter of the bed
    lst = temp[7:10]
    temp.append(np.average(lst))
    temp.append(np.std(lst))
    #Zone 3 the perimeter of the room
    lst = temp[1:7]
    lst.append(temp[14])
    temp.append(np.average(lst))
    temp.append(np.std(lst))
    #Zone 4 is just the outside sensor
    temp.append(temp[13])
    temp.append(1)
    dfMerged.append(temp)
columns.insert(0,'Date_Time')

In [15]:
mergedData = pd.DataFrame(dfMerged,columns = columns)

### Export Merged Frames

In [16]:
location = os.path.join('./mergedData/mergedFrame.csv')
mergedData.to_csv(location,index=False)

### Create csv files for each animation
We have 3 expirements in each that we want to average across the range

In [17]:
expTRange = {
    'OR7 Unblocked':
    [pd.Timestamp('12-22-2020 13:08:00'),
    pd.Timestamp('12-22-2020 13:16:30'),
    pd.Timestamp('12-22-2020 13:34:15')],
    'OR7 Blocked':
    [pd.Timestamp('12-22-2020 13:44:30'),
    pd.Timestamp('12-22-2020 13:53:00'),
    pd.Timestamp('12-22-2020 13:59:00')],
    'OR16 Unblocked':
    [pd.Timestamp('12-22-2020 14:38:00'),
    pd.Timestamp('12-22-2020 14:44:00'),
    pd.Timestamp('12-22-2020 14:50:00')],
    'OR16 Blocked 1':
    [pd.Timestamp('12-22-2020 14:58:30'),
    pd.Timestamp('12-22-2020 15:05:15'),
    pd.Timestamp('12-22-2020 15:11:00')],
    'OR16 Blocked 2':
    [pd.Timestamp('12-22-2020 15:17:30'),
    pd.Timestamp('12-22-2020 15:23:00'),
    pd.Timestamp('12-22-2020 15:30:00')],
}

In [18]:
# mergedData = pd.read_csv('./mergedData/mergedFrame.csv',parse_dates=[0])

In [19]:
time = mergedData['Date_Time']
expIndexes = {}
for i in expTRange:
    expIndexes[i] = []
    for x in expTRange[i]:
        for start,n in enumerate(time):
           if n >= x:
               expIndexes[i].append(start)
               break

In [32]:
indexRange = 24
averagedFrame = {}
expirementFrame = {}
for label in expIndexes:
    df1 = mergedData.iloc[expIndexes[label][0]-6:expIndexes[label][0]+indexRange,1:].reset_index(drop = True)
    df2 = mergedData.iloc[expIndexes[label][1]-6:expIndexes[label][1]+indexRange,1:].reset_index(drop = True)
    df3 = mergedData.iloc[expIndexes[label][2]-6:expIndexes[label][2]+indexRange,1:].reset_index(drop = True)
    averagedFrame[label] = (df1 + df2 + df3)/3
    expirementFrame[label+' Exp1'] = df1
    expirementFrame[label+' Exp2'] = df2
    expirementFrame[label+' Exp3'] = df3


In [34]:
for x in averagedFrame:
    temp=averagedFrame[x]
    location = os.path.join('./averagedData',x+'.csv')
    temp.to_csv(location,index=False)
for x in expirementFrame:
    temp=expirementFrame[x]
    location = os.path.join('./expirementData',x+'.csv')
    temp.to_csv(location,index=False)

### Increase the Resolution
pad out the dataframes to have values for every second.

In [36]:
stretchedDF = {}
for i in averagedFrame:
    tempFrame = averagedFrame[i].values
    tempList = []
    for idx,x in enumerate(tempFrame):
        try:
            increment = (tempFrame[idx+1] - x)/10
            for count in range(10):
                tempList.append(x+increment*count)
        except IndexError:
            tempList.append(x)
            continue
    stretchedDF[i] = pd.DataFrame(tempList, columns = expirementFrame['OR7 Unblocked Exp1'].columns)

stretchExpDf = {}
for i in expirementFrame:
    tempFrame = expirementFrame[i].values
    tempList = []
    for idx,x in enumerate(tempFrame):
        try:
            increment = (tempFrame[idx+1] - x)/10
            for count in range(10):
                tempList.append(x+increment*count)
        except IndexError:
            tempList.append(x)
            continue
    stretchExpDf[i] = pd.DataFrame(tempList, columns = expirementFrame['OR7 Unblocked Exp1'].columns)
        

In [37]:
for x in stretchedDF:
    temp=stretchedDF[x]
    location = os.path.join('./stretchedAvgData',x+'.csv')
    temp.to_csv(location,index=False)
for x in stretchExpDf:
    temp=stretchExpDf[x]
    location = os.path.join('./stretchedExpirementData',x+'.csv')
    temp.to_csv(location,index=False)